# ViT 모델 Fine-tuning 및 평가

이 노트북에서는 Vision Transformer 모델을 Fine-tuning하고 성능을 평가합니다.

## 목표
- ViT 모델 Fine-tuning
- 성능 지표 기록 (Accuracy, F1-score, Confusion Matrix)
- Loss Curve 시각화
- 오분류 샘플 분석


In [ ]:
# 필요한 라이브러리 import
import sys
sys.path.insert(0, '../src')

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
from PIL import Image
from tqdm import tqdm

# 프로젝트 모듈 import
from models.model_utils import create_vit_model, print_model_info
from data.dataset import ImageDataset
from data.preprocess import get_train_transforms, get_val_transforms
from training.loss_functions import get_loss_function
from training.optimizer import create_optimizer, create_scheduler
from training.train import train_model, EarlyStopping
from training.metrics import calculate_all_metrics, print_metrics, plot_confusion_matrix
from training.evaluator import evaluate_model
from utils.seed import set_seed

print("✅ 라이브러리 로드 완료!")


## 설정 및 데이터 준비


In [ ]:
# 시드 고정
set_seed(42)

# 경로 설정
DATA_DIR = Path("../data")
TRAIN_DIR = DATA_DIR / "train"
VAL_DIR = DATA_DIR / "val"
TEST_DIR = DATA_DIR / "test"
CHECKPOINT_DIR = Path("../experiments/checkpoints")
LOG_DIR = Path("../experiments/logs")
RESULTS_DIR = Path("../experiments/results")

# 결과 디렉토리 생성
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# 설정
CONFIG = {
    'model_name': 'vit_base',  # 'vit_base', 'vit_for_classification'
    'num_classes': 2,
    'batch_size': 16,  # ViT는 메모리를 더 많이 사용하므로 작은 배치 크기
    'num_epochs': 30,
    'learning_rate': 1e-5,  # Fine-tuning을 위해 작은 학습률
    'weight_decay': 1e-4,
    'image_size': 224,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'class_names': ['real', 'fake'],
    'class_map': {'real': '0', 'fake': '1'},
    'freeze_backbone': False,  # True로 설정하면 일부 레이어 고정
    'freeze_layers': None  # 고정할 레이어 수 (None이면 자동)
}

print(f"디바이스: {CONFIG['device']}")
print(f"모델: {CONFIG['model_name']}")
print(f"배치 크기: {CONFIG['batch_size']}")
print(f"에포크 수: {CONFIG['num_epochs']}")
print(f"Fine-tuning 모드: freeze_backbone={CONFIG['freeze_backbone']}")


In [ ]:
# 데이터셋 및 데이터로더 생성
train_transform = get_train_transforms(CONFIG['image_size'])
val_transform = get_val_transforms(CONFIG['image_size'])
test_transform = get_val_transforms(CONFIG['image_size'])

# 데이터셋 생성
train_dataset = ImageDataset(TRAIN_DIR, CONFIG['class_map'], transform=train_transform)
val_dataset = ImageDataset(VAL_DIR, CONFIG['class_map'], transform=val_transform)
test_dataset = ImageDataset(TEST_DIR, CONFIG['class_map'], transform=test_transform)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=4)

print(f"훈련 데이터: {len(train_dataset)}개")
print(f"검증 데이터: {len(val_dataset)}개")
print(f"테스트 데이터: {len(test_dataset)}개")


## 모델 생성


In [ ]:
# ViT 모델 생성 (Fine-tuning 구조)
model = create_vit_model(
    model_name=CONFIG['model_name'],
    num_classes=CONFIG['num_classes'],
    pretrained=True,
    freeze_backbone=CONFIG['freeze_backbone'],
    freeze_layers=CONFIG['freeze_layers']
)

model = model.to(CONFIG['device'])
print_model_info(model)


## Loss, Optimizer, Scheduler 설정


In [ ]:
# Loss 함수
criterion = get_loss_function('cross_entropy')

# Optimizer
optimizer = create_optimizer(
    model,
    'adamw',
    learning_rate=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay']
)

# Scheduler
scheduler = create_scheduler(
    optimizer,
    'reduce_lr_on_plateau',  # ViT는 ReduceLROnPlateau가 더 효과적
    patience=5,
    factor=0.5
)

# Early Stopping
early_stopping = EarlyStopping(patience=10, min_delta=0.001)


## 모델 Fine-tuning


In [ ]:
# Fine-tuning 실행
model_name = f"ViT_{CONFIG['model_name']}"
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    num_epochs=CONFIG['num_epochs'],
    device=CONFIG['device'],
    class_names=CONFIG['class_names'],
    save_dir=CHECKPOINT_DIR,
    log_dir=LOG_DIR,
    model_name=model_name,
    early_stopping=early_stopping,
    scheduler_name='reduce_lr_on_plateau'
)


## Loss Curve 시각화


In [ ]:
# Loss Curve 그리기
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss Curve
axes[0].plot(history['train_loss'], label='Train Loss', color='blue')
axes[0].plot(history['val_loss'], label='Val Loss', color='red')
axes[0].set_title('Loss Curve', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy Curve
axes[1].plot(history['train_accuracy'], label='Train Accuracy', color='blue')
axes[1].plot(history['val_accuracy'], label='Val Accuracy', color='red')
axes[1].set_title('Accuracy Curve', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / f'{model_name}_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ 그래프 저장: {RESULTS_DIR / f'{model_name}_curves.png'}")


## 테스트 세트 평가


In [ ]:
# Best 모델 로드
best_model_path = CHECKPOINT_DIR / f'{model_name}_best.pth'
checkpoint = torch.load(best_model_path, map_location=CONFIG['device'])
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"✅ Best 모델 로드 완료 (Epoch {checkpoint['epoch']})")

# 테스트 세트 평가
test_results = evaluate_model(
    model,
    test_loader,
    CONFIG['device'],
    class_names=CONFIG['class_names']
)

# 메트릭 출력
print_metrics(test_results['metrics'], class_names=CONFIG['class_names'])

# 결과 저장
test_metrics = {
    'accuracy': test_results['metrics']['accuracy'],
    'precision': test_results['metrics']['precision'],
    'recall': test_results['metrics']['recall'],
    'f1': test_results['metrics']['f1'],
    'confusion_matrix': test_results['metrics']['confusion_matrix'].tolist()
}

with open(RESULTS_DIR / f'{model_name}_test_metrics.json', 'w', encoding='utf-8') as f:
    json.dump(test_metrics, f, indent=2, ensure_ascii=False)


## Confusion Matrix 시각화


In [ ]:
# Confusion Matrix 시각화
cm = test_results['metrics']['confusion_matrix']
plot_confusion_matrix(
    cm,
    CONFIG['class_names'],
    save_path=RESULTS_DIR / f'{model_name}_confusion_matrix.png'
)


## 오분류 샘플 분석


In [ ]:
# 오분류 샘플 찾기
model.eval()
misclassified_samples = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="오분류 샘플 찾는 중"):
        images = images.to(CONFIG['device'])
        labels = labels.to(CONFIG['device'])
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        # 오분류 샘플 찾기
        incorrect = (predicted != labels).cpu().numpy()
        for i in range(len(images)):
            if incorrect[i]:
                misclassified_samples.append({
                    'image': images[i].cpu(),
                    'true_label': labels[i].item(),
                    'pred_label': predicted[i].item(),
                    'confidence': torch.softmax(outputs[i], dim=0)[predicted[i]].item()
                })

print(f"총 오분류 샘플 수: {len(misclassified_samples)}개")

# 오분류 샘플 시각화 (최대 16개)
num_samples = min(16, len(misclassified_samples))
if num_samples > 0:
    fig, axes = plt.subplots(4, 4, figsize=(16, 16))
    axes = axes.flatten()
    
    for idx in range(num_samples):
        sample = misclassified_samples[idx]
        img = sample['image']
        true_label = CONFIG['class_names'][sample['true_label']]
        pred_label = CONFIG['class_names'][sample['pred_label']]
        conf = sample['confidence']
        
        # 이미지 정규화 해제 및 표시
        img_denorm = img.permute(1, 2, 0)
        img_denorm = torch.clamp(img_denorm, 0, 1)
        
        axes[idx].imshow(img_denorm.numpy())
        axes[idx].set_title(f'True: {true_label}\\nPred: {pred_label} ({conf:.2f})',
                           fontsize=10, color='red' if true_label != pred_label else 'green')
        axes[idx].axis('off')
    
    # 빈 subplot 숨기기
    for idx in range(num_samples, len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle(f'Misclassified Samples ({num_samples}개)', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / f'{model_name}_misclassified.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ 오분류 샘플 저장: {RESULTS_DIR / f'{model_name}_misclassified.png'}")
else:
    print("오분류 샘플이 없습니다!")


## 결과 요약


In [ ]:
print("=" * 70)
print("📊 ViT 모델 Fine-tuning 결과 요약")
print("=" * 70)
print(f"모델: {CONFIG['model_name']}")
print(f"Best Epoch: {history['best_epoch']}")
print(f"Best Val Loss: {history['best_val_loss']:.4f}")
print(f"Best Val Accuracy: {history['best_val_accuracy']:.4f}")
print(f"\\n테스트 세트 성능:")
print(f"  Accuracy:  {test_metrics['accuracy']:.4f} ({test_metrics['accuracy']*100:.2f}%)")
print(f"  Precision: {test_metrics['precision']:.4f}")
print(f"  Recall:    {test_metrics['recall']:.4f}")
print(f"  F1 Score:  {test_metrics['f1']:.4f}")
print(f"\\n저장된 파일:")
print(f"  - 체크포인트: {CHECKPOINT_DIR / f'{model_name}_best.pth'}")
print(f"  - Loss Curve: {RESULTS_DIR / f'{model_name}_curves.png'}")
print(f"  - Confusion Matrix: {RESULTS_DIR / f'{model_name}_confusion_matrix.png'}")
print(f"  - Misclassified: {RESULTS_DIR / f'{model_name}_misclassified.png'}")
print(f"  - 메트릭: {RESULTS_DIR / f'{model_name}_test_metrics.json'}")
print("=" * 70)
